In [1]:
%reload_ext autoreload
%autoreload 2
%cd ..

/Users/anton.javelosa/Desktop/Personal/Personal Projects/taxi_demand_predictor


/Users/anton.javelosa/Desktop/Personal/Personal Projects/taxi_demand_predictor/.venv/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import src.config as config

In [3]:
import hopsworks

# connect to the project
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

# connect to the feature store
feature_store = project.get_feature_store()

# connect to the feature group
feature_group = feature_store.get_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
)

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/760982
Connected. Call `.close()` to terminate connection gracefully.


In [5]:
# create feature view if it doesn't exist yet
# This feature view only uses the features from the feature group so the query is trivial
try:
    # create feature view if it doesn't exist yet
    feature_store.create_feature_view(
        name=config.FEATURE_VIEW_NAME,
        version=config.FEATURE_VIEW_VERSION,
        query=feature_group.select_all()
    )
except:
    print("Feature view already exists. Skip creation")

# get feature view
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME,
    version=config.FEATURE_VIEW_VERSION
)

Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/760982/fs/756805/fv/time_series_hourly_feature_group/version/1


In [6]:
ts_data, __ = feature_view.training_data(
    description='Time series hourly taxi rides',
)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (312.06s) 
2024-05-31 21:14:34,500 WARNING: VersionWarning: Incremented version to `1`.



In [7]:
ts_data.sort_values(by=['pickup_location_id', 'pickup_hour'], inplace=True)
ts_data.head(3)

,pickup_hour,rides,pickup_location_id,pickup_ts
802612,2022-01-01 00:00:00+00:00,0,1,1640995200000
4138081,2022-01-01 01:00:00+00:00,0,1,1640998800000
117519,2022-01-01 02:00:00+00:00,0,1,1641002400000


In [9]:
ts_data.drop(columns=['pickup_ts'], inplace=True)

In [15]:
# ignore deprecation warnings for now

import warnings
warnings.filterwarnings("ignore")

In [16]:
from src.data import transform_ts_data_into_features_and_target

features, targets = transform_ts_data_into_features_and_target(
    ts_data,
    input_seq_len=24*28, # 1 month
    step_size=23,
)

features_and_target = features.copy()
features_and_target['target_rides_next_hour'] = targets

print(f'{features_and_target.shape=}')

100%|██████████| 265/265 [00:30<00:00,  8.80it/s]


features_and_target.shape=(217469, 675)


In [31]:
from datetime import date, datetime, timedelta
from pytz import timezone
import pandas as pd
from src.data_split import train_test_split

# training data from Jan 2022 -> 2 months ago
# test data -> last 2 months
cutoff_date = pd.to_datetime(datetime.today() - timedelta(days = 28*1)).date()
print(f'{date.today() - timedelta(days = 28*1)=}')
print(f'{cutoff_date=}')

X_train, y_train, X_test, y_test = train_test_split(
    features_and_target,
    cutoff_date=cutoff_date,
    target_column_name='target_rides_next_hour',
)

print(f'{X_train.shape=} {y_train.shape=}')
print(f'{X_test.shape=} {y_test.shape=}')

date.today() - timedelta(days = 28*1)=datetime.date(2024, 5, 4)
cutoff_date=datetime.date(2024, 5, 4)


TypeError: '<' not supported between instances of 'str' and 'datetime.date'